# Risk Pooling
Simulate the risk pooling game as discussed in the lecture. For this problem as well, assume that one time step corresponds to one week. Consider two scenarios. One with two warehouses catering to the two different products (A,B) and the second scenario corresponding to a centralized warehouse. 

- Assume the demand for products A and B for both warehouses in scenario one comes from normal distributions N(40,12) and N(1,1) respectively. 
- Assume that the demand for the centralized warehouse in scenario 2 comes from N(80,20) and N(2, 2) respectively.
- Assume a random lead time of [1, 2] weeks. (i.e. the lead time can be 1 or 2 weeks with equal probability).
- Assume the warehouses use the (s, S) policy to manage inventory. Assume that the starting inventory for both warehouses in scenario 1 is 50 units and 2 units for products A and B respectively and for scenario 2 2, it is 80 units and 3 units respectively.

Run this scenario for 100 time steps.
1. Find out the stockout rate (fraction of time periods) in scenario 1 as well as scenario 2.
2. Compare the total holding inventory in both the scenarios over the toal time.(use the same parameters for holding cost, ordering cost and service rate as mentioned in the lecture notes.)
- maintain 97% service level 
- $60 order cost
- \$.27 weekly holding cost 

In [ ]:
import simpy
import numpy as np
import pandas as pd
import random

In [ ]:
# Set seed for repeatability
np.random.seed(7)
random.seed(7)

# Constants
lead_time = [1, 2]
mean_lead_time = np.mean(lead_time)
sd_lead_time = np.std(lead_time)
time_steps = 100

order_cost = 60
holding_cost = 0.27
z_alpha = 2.17  # for 97% service level

# Functions
def calculate_s(mean_demand, stddev_demand):
    return mean_demand * mean_lead_time + z_alpha * np.sqrt(mean_demand**2 * mean_lead_time + stddev_demand**2 * sd_lead_time**2)

def calculate_S(D):
    return order_cost + np.sqrt(2 * D * order_cost / holding_cost)

def check_demand(demand_mean, demand_std_dev):
    return max(0, np.random.normal(demand_mean, demand_std_dev))

# Simulate an inventory system
# Simulate an inventory system
def inventory_system(env, inventory, demand_mean, demand_std_dev, output):
    total_held_inventory = 0
    stockouts = 0

    for _ in range(time_steps):
        demand = check_demand(demand_mean, demand_std_dev)
        actual_demand = min(inventory.level, demand)
        
        if actual_demand > 0:  # only get from inventory when actual_demand is > 0
            yield inventory.get(actual_demand)

        if actual_demand < demand:
            stockouts += 1

        yield env.timeout(1)

        if inventory.level < calculate_s(demand_mean, demand_std_dev):
            env.process(order_stock(env, inventory, calculate_S(demand_mean)))

        total_held_inventory += inventory.level

    output["stockout_rate"] = stockouts / time_steps
    output["total_holding_cost"] = total_held_inventory * holding_cost

# Order stock
def order_stock(env, inventory, S):
    order_qty = max(0, S - inventory.level)
    if order_qty > 0:
        yield env.timeout(random.choice(lead_time))
        yield inventory.put(order_qty)

# Parameters
scenarios = [
    ('Decentralized', 50, 2, 40, 12, 1, 1),
    ('Centralized', 80, 3, 80, 20, 2, 2),
]

# Run simulations
results = []

for scenario_name, starting_inventory_A, starting_inventory_B, demand_mean_A, demand_sd_A, demand_mean_B, demand_sd_B in scenarios:
    output_A = {}
    env_A = simpy.Environment()
    inventory_A = simpy.Container(env_A, init=starting_inventory_A)
    env_A.process(inventory_system(env_A, inventory_A, demand_mean_A, demand_sd_A, output_A))
    env_A.run()

    output_B = {}
    env_B = simpy.Environment()
    inventory_B = simpy.Container(env_B, init=starting_inventory_B)
    env_B.process(inventory_system(env_B, inventory_B, demand_mean_B, demand_sd_B, output_B))
    env_B.run()

    results.append({
        'Scenario': scenario_name,
        'Stockout Rate Product A': output_A["stockout_rate"],
        'Stockout Rate Product B': output_B["stockout_rate"],
        'Total Holding Cost Product A': output_A["total_holding_cost"],
        'Total Holding Cost Product B': output_B["total_holding_cost"],
    })

# Create DataFrame
results_df = pd.DataFrame(results)
print(results_df)

        Scenario  Stockout Rate Product A  Stockout Rate Product B  \
0  Decentralized                     0.03                     0.01   
1    Centralized                     0.02                     0.01   

   Total Holding Cost Product A  Total Holding Cost Product B  
0                   4810.551287                   2962.440641  
1                   6849.347951                   2395.111652  
